In [2]:
from ast import literal_eval
from csv import reader
from os import listdir, makedirs, path
from pickle import dump
import pickle
import numpy as np
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from model import Seq2Seq, RecurrentAutoencoder, DNNAE, SalTransformer, SalAE, SalSCINet, SalGATSCINet, SalGATSCINetV2, SalGATConvLSTM, SalGATConvGRU, ConvGRU, SalGATConvGRUwoSal, SalConvGRUwoALL, SalGATConvGRUwoGAT


import os, random
import torch
from torch.nn import TransformerAE
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)



def load_and_save(category, filename, dataset, dataset_folder, output_folder):
    temp = np.genfromtxt(
        path.join(dataset_folder, category, filename),
        dtype=np.float32,
        delimiter=",",
    )
    print(dataset, category, filename, temp.shape)
    with open(path.join(output_folder, dataset + "_" + category + ".pkl"), "wb") as file:
        dump(temp, file)


def load_data(dataset):
    """ Method from OmniAnomaly (https://github.com/NetManAIOps/OmniAnomaly) """

    if dataset == "SMD":
        dataset_folder = "/home/sangyup/saliency_anomaly_detection/dataset/ServerMachineDataset"
        output_folder = "/home/sangyup/saliency_anomaly_detection/dataset/ServerMachineDataset/processed"
        makedirs(output_folder, exist_ok=True)
        file_list = listdir(path.join(dataset_folder, "train"))
        for filename in file_list:
            if filename.endswith(".txt"):
                load_and_save(
                    "train",
                    filename,
                    filename.strip(".txt"),
                    dataset_folder,
                    output_folder,
                )
                load_and_save(
                    "test_label",
                    filename,
                    filename.strip(".txt"),
                    dataset_folder,
                    output_folder,
                )
                load_and_save(
                    "test",
                    filename,
                    filename.strip(".txt"),
                    dataset_folder,
                    output_folder,
                )

    elif dataset == "SMAP" or dataset == "MSL":
        # Change the dataset path accordingly
        dataset_folder = "/SMAPMSL/data"
        output_folder = "/SMAPMSL/data/processed"
        makedirs(output_folder, exist_ok=True)
        with open(path.join(dataset_folder, "labeled_anomalies.csv"), "r") as file:
            csv_reader = reader(file, delimiter=",")
            res = [row for row in csv_reader][1:]
        res = sorted(res, key=lambda k: k[0])
        data_info = [row for row in res if row[1] == dataset and row[0] != "P-2"]
        labels = []
        for row in data_info:
            anomalies = literal_eval(row[2])
            length = int(row[-1])
            label = np.zeros([length], dtype=np.bool_)
            for anomaly in anomalies:
                label[anomaly[0] : anomaly[1] + 1] = True
            labels.extend(label)

        labels = np.asarray(labels)
        print(dataset, "test_label", labels.shape)

        with open(path.join(output_folder, dataset + "_" + "test_label" + ".pkl"), "wb") as file:
            dump(labels, file)

        def concatenate_and_save(category):
            data = []
            for row in data_info:
                filename = row[0]
                temp = np.load(path.join(dataset_folder, category, filename + ".npy"))
                data.extend(temp)
            data = np.asarray(data)
            print(dataset, category, data.shape)
            with open(path.join(output_folder, dataset + "_" + category + ".pkl"), "wb") as file:
                dump(data, file)

        for c in ["train", "test"]:
            concatenate_and_save(c)

def normalize_data(data, scaler=None):
    data = np.asarray(data, dtype=np.float32)
    if np.any(sum(np.isnan(data))):
        data = np.nan_to_num(data)

    if scaler is None:
        scaler = MinMaxScaler()
        scaler.fit(data)
    data = scaler.transform(data)
    print("Data normalized")

    return data, scaler

def get_data_dim(dataset):
    """
    :param dataset: Name of dataset
    :return: Number of dimensions in data
    """
    if dataset == "SMAP":
        return 25
    elif dataset == "MSL":
        return 55
    elif str(dataset).startswith("machine"):
        return 38
    else:
        raise ValueError("unknown dataset " + str(dataset))

        
def get_data(dataset, max_train_size=None, max_test_size=None,
             normalize=False, spec_res=False, train_start=0, test_start=0):
    """
    Get data from pkl files

    return shape: (([train_size, x_dim], [train_size] or None), ([test_size, x_dim], [test_size]))
    Method from OmniAnomaly (https://github.com/NetManAIOps/OmniAnomaly)
    """
    prefix = "/home/sangyup/saliency_anomaly_detection/dataset"
    if str(dataset).startswith("machine"):
        prefix += "/ServerMachineDataset/processed"
    elif dataset in ["MSL", "SMAP"]:
        prefix += "/SMAPMSL/data/processed"
    if max_train_size is None:
        train_end = None
    else:
        train_end = train_start + max_train_size
    if max_test_size is None:
        test_end = None
    else:
        test_end = test_start + max_test_size
    print("load data of:", dataset)
    print("train: ", train_start, train_end)
    print("test: ", test_start, test_end)
    x_dim = get_data_dim(dataset)
    f = open(os.path.join(prefix, dataset + "_train.pkl"), "rb")
    train_data = pickle.load(f).reshape((-1, x_dim))[train_start:train_end, :]
    f.close()
    try:
        f = open(os.path.join(prefix, dataset + "_test.pkl"), "rb")
        test_data = pickle.load(f).reshape((-1, x_dim))[test_start:test_end, :]
        f.close()
    except (KeyError, FileNotFoundError):
        test_data = None
    try:
        f = open(os.path.join(prefix, dataset + "_test_label.pkl"), "rb")
        test_label = pickle.load(f).reshape((-1))[test_start:test_end]
        f.close()
    except (KeyError, FileNotFoundError):
        test_label = None

    if normalize:
        train_data, scaler = normalize_data(train_data, scaler=None)
        test_data, _ = normalize_data(test_data, scaler=scaler)

    print("train set shape: ", train_data.shape)
    print("test set shape: ", test_data.shape)
    print("test set label shape: ", None if test_label is None else test_label.shape)
    return (train_data, None), (test_data, test_label)


def split_series(series, n_past, n_future):
    '''

    :param series: input time series
    :param n_past: number of past observations
    :param n_future: number of future series
    :return: X, y(label)
    '''
    X, y = list(), list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        # slicing the past and future parts of the window
        past, future = series[window_start:past_end, :], series[past_end:future_end, :]
        X.append(past)
        y.append(future)

    return X, y

In [4]:
ds = 'MSL'  # SMD / MSL / SMD
load_data(ds.upper())

# SMD 1-1, 1-2, 1-3
# (x_train, _), (x_test, y_test) = get_data('machine-1-1', normalize=True)

# MSL / SMAP
# (x_train, _), (x_test, y_test) = get_data(ds, normalize=True)


load data of: MSL
train:  0 None
test:  0 None
Data normalized
Data normalized
train set shape:  (58317, 55)
test set shape:  (73729, 55)
test set label shape:  (73729,)


In [6]:
from torch.utils.data import Dataset, DataLoader
class ReconDataset(Dataset):
    def __init__(self, data, window, target_cols):
        self.data = torch.Tensor(data)
        self.window = window
        self.target_cols = target_cols
        self.shape = self.__getshape__()
        self.size = self.__getsize__()

    def __getitem__(self, index):
        x = self.data[index:index+self.window]
        y = self.data[index:index+self.window]
        return x, y

    def __len__(self):
        return len(self.data) -  self.window 
    
    def __getshape__(self):
        return (self.__len__(), *self.__getitem__(0)[0].shape)

    def __getsize__(self):
        return (self.__len__())

class ForecastDataset(Dataset):
    def __init__(self, data, window, target_cols):
        self.data = torch.Tensor(data)
        self.window = window
        self.target_cols = target_cols
        self.shape = self.__getshape__()
        self.size = self.__getsize__()

    def __getitem__(self, index):
        x = self.data[index:index+self.window]
        y = self.data[index+self.window,0:self.target_cols]
        return x, y

    def __len__(self):
        return len(self.data) -  self.window 
    
    def __getshape__(self):
        return (self.__len__(), *self.__getitem__(0)[0].shape)
    
    def __getsize__(self):
        return (self.__len__())

class ReconForecastDataset(Dataset):
    def __init__(self, data, window, horizon):
        self.data = torch.Tensor(data)
        self.window = window
        self.horizon = horizon
        self.shape = self.__getshape__()
        self.size = self.__getsize__()

    def __getitem__(self, index):
        x = self.data[index:index+self.window]
        y_recon = self.data[index:index+self.window]
        y_fore = self.data[index+self.window:index+self.window+self.horizon]
        return x, y_recon, y_fore

    def __len__(self):
        return len(self.data) -  self.window 
    
    def __getshape__(self):
        return (self.__len__(), *self.__getitem__(0)[0].shape), (self.__len__(), *self.__getitem__(0)[1].shape), (self.__len__(), *self.__getitem__(0)[2].shape)
    
    def __getsize__(self):
        return (self.__len__())


def create_data_loaders(train_dataset, batch_size, val_split=0.1, shuffle=False, test_dataset=None):
    train_loader, val_loader, test_loader = None, None, None
    if val_split == 0.0:
        print(f"train_size: {len(train_dataset)}")
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, drop_last=True)

    else:
        dataset_size = len(train_dataset)
        indices = list(range(dataset_size))
        split = int(np.floor(val_split * dataset_size))
        if shuffle:
            np.random.shuffle(indices)
        train_indices, val_indices = indices[split:], indices[:split]

        train_sampler = SubsetRandomSampler(train_indices)
        valid_sampler = SubsetRandomSampler(val_indices)

        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, sampler=train_sampler, drop_last=True)
        val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, sampler=valid_sampler, drop_last=True)

        print(f"train_size: {len(train_indices)}")
        print(f"validation_size: {len(val_indices)}")

    if test_dataset is not None:
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
        print(f"test_size: {len(test_dataset)}")

    return train_loader, val_loader, test_loader

In [6]:
window_size = 100   # Historical data lookback
batch_size = 512    # Batch size
horizon = 10        # Future prediction horizon length

train_dataset = ReconForecastDataset(x_train, window_size, horizon)
indices = torch.arange(len(train_dataset)-horizon)
train_dataset = torch.utils.data.Subset(train_dataset, indices)
test_dataset = ReconForecastDataset(x_test, window_size, horizon)

train_loader, val_loader, test_loader = create_data_loaders(train_dataset, batch_size, val_split=0.3, shuffle=False, test_dataset=test_dataset)

train_size: 40745
validation_size: 17462
test_size: 73629


In [7]:
from time import time

def train_model(dataloader, val_loader, model, batch_size, n_epochs, model_name):
    optimizer = torch.optim.AdamW(model.parameters())
    
    loss_fn_AE = torch.nn.MSELoss()
    loss_fn_sci = torch.nn.MSELoss()

    epochs = range(n_epochs)
    best = {"loss": sys.float_info.max}
    loss_history, loss_history_AE, loss_history_TF = [], [], []
    val_loss_history, val_loss_history_AE, val_loss_history_TF = [], [], []
    for e in epochs:
        model.train()
        start = time()
        epoch_loss = 0
        epoch_loss_AE = 0
        epoch_loss_sci = 0
        for i, (x,y_recon,y_fore) in enumerate(dataloader):  
            optimizer.zero_grad()
            x = x.cuda()
            y_recon = y_recon.cuda()
            y_fore = y_fore.cuda()
            sal, dec_x, out = model(x)
            
            loss_AE = loss_fn_AE(y_recon, dec_x)
            loss_sci = loss_fn_sci(y_fore, out)
            loss = loss_AE + loss_sci
            loss.backward()
            epoch_loss += loss.item()
            epoch_loss_AE += loss_AE.item()
            epoch_loss_sci += loss_sci.item()
            optimizer.step()
        loss_history.append(epoch_loss)
        loss_history_AE.append(epoch_loss_AE)
        loss_history_TF.append(epoch_loss_sci)

        
        # Validation step
        model.eval()
        val_epoch_loss = 0
        val_epoch_loss_AE = 0
        val_epoch_loss_sci = 0
        with torch.no_grad():
            for i, (x,y_recon,y_fore) in enumerate(val_loader):  
                x = x.cuda()
                y_recon = y_recon.cuda()
                y_fore = y_fore.cuda()
                sal, dec_x, out = model(x)

                valloss_AE = loss_fn_AE(y_recon, dec_x)
                valloss_sci = loss_fn_sci(y_fore, out)
                valloss = valloss_AE + valloss_sci
                
                val_epoch_loss += valloss.item()
                val_epoch_loss_AE += valloss_AE.item()
                val_epoch_loss_sci += valloss_sci.item()
                
                val_loss_history.append(val_epoch_loss)
                val_loss_history_AE.append(val_epoch_loss_AE)
                val_loss_history_TF.append(val_epoch_loss_sci)  

        print(f'Training loss EPOCH: [{e+1}|{len(epochs)}], training loss: [{epoch_loss}], AE loss: [{epoch_loss_AE}], TF loss: [{epoch_loss_sci}] took', time()-start)
        print(f'Validation loss EPOCH: [{e+1}|{len(epochs)}], validation loss: [{val_epoch_loss}], AE loss: [{val_epoch_loss_AE}], TF loss: [{val_epoch_loss_sci}]')
        if val_epoch_loss < best["loss"]:
            best["state"] = model.state_dict()
            best["loss"] = val_epoch_loss
            best["loss_AE"] = val_epoch_loss_AE
            best["loss_TF"] = val_epoch_loss_sci
            best["epoch"] = e + 1
            with open("result/model_" + model_name + "_best.pt", "wb") as f:
                torch.save(
                    {
                        "state": best["state"],
                        "best_epoch": best["epoch"],
                        "loss_history": val_loss_history,
                        "loss_AE_history": val_loss_history_AE,
                        "loss_TF_history": val_loss_history_TF,
                    },
                    f,
                )

    # Save last epoch
    with open("result/model_" + model_name + "_last.pt", "wb") as f:
        torch.save(
            {
                "state": model.state_dict(),
                "best_epoch": e + 1,
                "loss_history": val_loss_history,
                "loss_AE_history": val_loss_history_AE,
                "loss_TF_history": val_loss_history_TF,
            },
            f,
        )

    return best, loss_history, loss_history_AE, loss_history_TF
 

In [8]:

MODEL = SalGATConvGRU(seq_len=window_size, output_len=10, n_features=x_train.shape[1], out_n_features=x_train.shape[1], embedding_dim=int(x_train.shape[1]/2), kernel_size=3, cell='gru')
# MODEL = SalGATConvGRUwoSal(seq_len=window_size, output_len=10, n_features=x_train.shape[1], out_n_features=x_train.shape[1], embedding_dim=int(x_train.shape[1]/2), kernel_size=3, cell='gru')
# MODEL = SalConvGRUwoALL(seq_len=window_size, output_len=10, n_features=x_train.shape[1], out_n_features=x_train.shape[1], embedding_dim=int(x_train.shape[1]/2), kernel_size=3, cell='gru')
MODEL.cuda()
MODEL.train()

SalConvGRUwoALL(
  (encoder): RecurEncoder(
    (rnn1): GRU(55, 54, batch_first=True)
    (rnn2): GRU(54, 27, batch_first=True)
  )
  (decoder): RecurDecoder(
    (rnn1): GRU(27, 27, batch_first=True)
    (rnn2): GRU(27, 54, batch_first=True)
    (output_layer): Linear(in_features=54, out_features=55, bias=True)
    (timedist): TimeDistributed(
      (module): Linear(in_features=54, out_features=55, bias=True)
    )
  )
  (feature_gat): FeatureAttentionLayer(
    (lin): Linear(in_features=100, out_features=100, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
    (sigmoid): Sigmoid()
  )
  (temporal_gat): TemporalAttentionLayer(
    (lin): Linear(in_features=55, out_features=55, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
    (sigmoid): Sigmoid()
  )
  (convGRU): ConvGRU(
    (cell_list): ModuleList(
      (0): ConvGRUCell(
        (conv_gates): Conv2d(87, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv_can): Conv2d(87, 32, kernel_size=(3, 3)

In [ ]:
model_name = 'SalAD_'+ds
epochs = 1000
BEST_MODEL, LOSS_HISTORY, LOSS_HISTORY_AE, LOSS_HISTORY_TF = train_model(train_loader, val_loader, MODEL, batch_size, epochs, model_name)

In [ ]:
model_name = 'SalAD_MSL'
with open("result/model_" + model_name + "_best.pt", "rb") as f:
    SAVED_MODEL = torch.load(f)

print(SAVED_MODEL['best_epoch'])


MODEL = SalGATConvGRU(seq_len=window_size, output_len=10, n_features=x_train.shape[1], out_n_features=x_train.shape[1], embedding_dim=int(x_train.shape[1]/2), kernel_size=3, cell='gru')
# MODEL = SalGATConvGRUwoSal(seq_len=window_size, output_len=10, n_features=x_train.shape[1], out_n_features=x_train.shape[1], embedding_dim=int(x_train.shape[1]/2), kernel_size=3, cell='gru')
# MODEL = SalConvGRUwoALL(seq_len=window_size, output_len=10, n_features=x_train.shape[1], out_n_features=x_train.shape[1], embedding_dim=int(x_train.shape[1]/2), kernel_size=3, cell='gru')

MODEL.cuda()    
MODEL.load_state_dict(SAVED_MODEL["state"])

In [11]:
def inference_SAL(dataloader, model, batch_size, TF_alpha):
    dist, dist_sal, fin_dist1, fin_dist2, guess, sal_list = [], [], [], [], [], []
    mse = torch.nn.MSELoss()
    model.eval()
    with torch.no_grad():
        for i, (x,y_recon,y_fore) in enumerate(dataloader):
            x = x.cuda()
            y_recon = y_recon.cuda()
            y_fore = y_fore.cuda()
            
            sal, dec_x, out = model(x)
            
            for y_r, y_f, d, o in zip(y_recon, y_fore, dec_x, out):
                d_s = torch.sum(torch.square(y_r - d)).cpu().numpy()
                d_o = torch.sum(torch.square(y_f - o)).cpu().numpy()
                # d_s = torch.mean(torch.square(y_r - d)).cpu().numpy()
                # d_o = torch.mean(torch.square(y_f - o)).cpu().numpy()
                
                dist_sal.append(d_s)
                dist.append(d_o)

                # inference_score1 = (d_s + d_o)/(1+TF_alpha)
                inference_score1 = ((1-TF_alpha)*d_s + TF_alpha*d_o)/(1+TF_alpha)
                inference_score2 = (d_s + TF_alpha*d_o)/(1+TF_alpha)
                fin_dist1.append(inference_score1)
                fin_dist2.append(inference_score2)
                # fin_dist.append((1-TF_alpha)*mse(y_s,d).item() + TF_alpha*mse(y_s, o).item())
                # fin_dist.append((TF_alpha*torch.cdist(y_s, o, p=2.0) + (1-TF_alpha)*torch.cdist(y_s, d, p=2.0)).cpu().numpy())

            guess.append(out.cpu().numpy())
            sal_list.append(sal.cpu().numpy())
        
            
    return (
        dist,
        dist_sal,
        fin_dist1,
        fin_dist2,
        np.concatenate(guess),
        np.concatenate(sal_list)
    )

In [12]:
%%time
MODEL.eval()
DIST, DIST_SAL, FIN_DIST1, FIN_DIST2, GUESS, SAL_LIST = inference_SAL(test_loader, MODEL, batch_size, TF_alpha=1.0)
DIST_train, DIST_SAL_train, FIN_DIST1_train, FIN_DIST2_train, GUESS_train, SAL_LIST_train = inference_SAL(train_loader, MODEL, batch_size, TF_alpha=1.0)

CPU times: user 20min 19s, sys: 33.4 s, total: 20min 52s
Wall time: 29.2 s


In [ ]:
%%time
from sklearn import metrics

MODEL.eval()
alpha = 1.0
DIST, DIST_SAL, FIN_DIST1, FIN_DIST2, GUESS, SAL_LIST = inference_SAL(test_loader, MODEL, batch_size, TF_alpha=alpha)
DIST_train, DIST_SAL_train, FIN_DIST1_train, FIN_DIST2_train, GUESS_train, SAL_LIST_train = inference_SAL(train_loader, MODEL, batch_size, TF_alpha=alpha)

# anomaly score of test dataset
res = SAL_LIST[:,-1:,:]
res = res.reshape(res.shape[0], res.shape[2])
res_mean = np.mean(res, axis=1)
FD1wSAL = np.array(FIN_DIST1)*res_mean

# anomaly score of train dataset --> to calculate the threshold
res_train = SAL_LIST_train[:,-1:,:]
res_train = res_train.reshape(res_train.shape[0], res_train.shape[2])
res_mean_train = np.mean(res_train, axis=1)
FD1wSAL_train = np.array(FIN_DIST1_train)*res_mean_train


fpr, tpr, thresholds = metrics.roc_curve(y_test[-len(FD1wSAL):].astype(int), FD1wSAL, pos_label=1)
metrics.auc(fpr, tpr)